## untested,maybe uncorrected 

In [1]:
import numpy as np
import struct,os
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros
import torch

def load_mnist(image_file, label_file):
    digits=np.arange(10)

    fname_image = os.path.join("../"+image_file)
    fname_label = os.path.join("../"+label_file)

    flbl = open(fname_label, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_image, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows*cols), dtype=uint8)
    labels = zeros((N, 1), dtype=int8)
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((1, rows*cols))
        labels[i] = lbl[ind[i]]

    return images, labels

train_image, train_label = load_mnist("train-images.idx3-ubyte", "train-labels.idx1-ubyte")
test_image, test_label = load_mnist("t10k-images.idx3-ubyte", "t10k-labels.idx1-ubyte")


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device ='cpu'
layers=4
lsize=[train_image.shape[1],19*19,18*18,15*15,10]

W=[(torch.rand(lsize[i],lsize[i+1],dtype=torch.float32)-0.5).to(device) for i in range(layers)]
b=[(torch.rand(lsize[i],dtype=torch.float32)-0.5).to(device) for i in range(1,layers+1)]

ty=torch.zeros([train_image.shape[0],lsize[len(lsize)-1]],dtype=torch.float32).to(device)
for i in range(train_image.shape[0]):
    ty[i][train_label[i]]=1.0

In [3]:
print("W's shape is")
for i in range(len(W)):
    print(W[i].shape)
print("b's shape is")
for i in range(len(b)):
    print(b[i].shape)

W's shape is
torch.Size([784, 361])
torch.Size([361, 324])
torch.Size([324, 225])
torch.Size([225, 10])
b's shape is
torch.Size([361])
torch.Size([324])
torch.Size([225])
torch.Size([10])


In [3]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_image=torch.from_numpy(train_image).float().to(device)
train_label=torch.from_numpy(train_label).int().to(device)
test_image=torch.from_numpy(test_image).float().to(device)
test_label=torch.from_numpy(test_label).int().to(device)

In [95]:
m = torch.nn.Sigmoid()
A=[]
def predict(img):
    A.clear()
    A.append(m(img@W[0]+b[0]))
    for i in range(layers-1):
        A.append(A[i]@W[i+1]+b[i+1])
    return A[len(A)-1]

In [6]:
print("A's shape is")
for i in range(len(A)):
    print(A[i].shape)

A's shape is


In [ ]:
def print_train_error():
    err=0
    Y=torch.argmax(predict(train_image),dim=1)
    for i in range(Y.shape[0]):
        if(Y[i]!=train_label[i]):
            err+=1
    print("%.10lf"%(1-err/train_image.shape[0]))
print_train_error()

In [ ]:
def print_genaralization_error():
    err=0
    Y=torch.argmax(predict(test_image),dim=1)
    for i in range(Y.shape[0]):
        if(Y[i]!=test_label[i]):
            err+=1
    print("%.10lf"%(1-err/test_image.shape[0]))
print_genaralization_error()

In [96]:
def loss():
    tmp=(predict(train_image)-ty)
    # print(tmp.shape)
    print((tmp*tmp).sum().item())
loss()

282136960.0


In [14]:
torch.set_printoptions(precision=10)

In [93]:
db=torch.load("model.pth")
for i in range(layers):
    W[i]=db["W"+str(i)]
for i in range(layers):
    b[i]=db["b"+str(i)]

In [ ]:
alpha=0.000000000001/train_image.shape[0]
T=20
while T>0:
    predict(train_image)
    dlt=[]
    n=len(A)-1
    dlt.append(-(ty-A[n])*A[n]*(1-A[n])) #? 
    for i in range(layers-1,0,-1):
        dlt.append((dlt[layers-i-1]@W[i].T)*A[i-1]*(1-A[i-1]))
    dlt.reverse()
    W[0]-=alpha*(train_image.T)@dlt[0]
    # print(dlt[0].sum(axis=0).shape)
    b[0]-=alpha*dlt[0].sum(axis=0)
    for i in range(1,layers):
        W[i]-=alpha*(A[i-1].T)@dlt[i]
        # print(dlt[i].sum(axis=0).shape)
        b[i]-=alpha*dlt[i].sum(axis=0)
    T-=1
    dlt.clear()
    if(T%1==0):
        # print_train_error()
        alpha*=1
        loss()
        print(b[3])
    

In [13]:
for i in range(len(dlt)):
    print(dlt[i].shape)

In [90]:
db={"W0":W[0],"W1":W[1],"W2":W[2],"W3":W[3],"b0":b[0],"b1":b[1],"b2":b[2],"b3":b[3]}
torch.save(db,"model.pth")

In [ ]:
from matplotlib import pyplot as plt
err=0
sum=[0 for i in range(10)]
ecnt=[0 for i in range(10)]
Y=torch.argmax(predict(test_image),dim=1)
for i in range(Y.shape[0]):
    sum[test_label[i]]+=1
    if(Y[i]!=test_label[i]):
        err+=1
        ecnt[test_label[i]]+=1
        if err<=50:
            plt.subplots_adjust(left=1, bottom=None, right=3, top=2, wspace=None, hspace=None)
            plt.subplot(5, 10, err)
            plt.axis('off')
            img=np.array(list(test_image[i].cpu())).reshape(28,-1)
            plt.imshow(img, cmap='gray',interpolation='nearest')
            plt.title("p:%d t:%d"% (Y[i],test_label[i]))
for i in range(len(sum)):
    print("%d:%d/%d=%.10lf"%(i,ecnt[i],sum[i],ecnt[i]/sum[i]))